In [1]:
import os

go_up_n_directories = lambda path, n: os.path.abspath(
    os.path.join(*([os.path.dirname(path)] + [".."] * n))
)
os.chdir(go_up_n_directories(os.getcwd(), 4))  # run once (otherwise restart kernel)
os.getcwd()

'/home/tim/Development/OCPPM'

In [2]:
# Python natives
import pprint
import pickle
import time
import json
import os
from utilities import evaluation_utils
from ocpa.algo.predictive_monitoring.obj import Feature_Storage
from ocpa.algo.predictive_monitoring import tabular

In [3]:
feature_storage_file = (
    "data/OTC/feature_encodings/EFG/efg/raw/OTC_split_[C2_P2_P3_O3_eas].fs"
)
target = ("event_remaining_time", ())
model_output_path = "models/OTC/baselines/median"

In [4]:
with open(feature_storage_file, "rb") as bin:
    feature_storage: Feature_Storage = pickle.load(bin)

In [5]:
# flatten EFG with same train/test split
eft_train = tabular.construct_table(
    feature_storage, feature_storage.train_indices + feature_storage.validation_indices
)
eft_test = tabular.construct_table(feature_storage, feature_storage.test_indices)

In [6]:
# subset relevant data
y_train = eft_train[target]
y_test = eft_test[target]

In [11]:
start_train_time = time.time()
# fit the model ;)
median = y_train.median()
total_train_time = time.time() - start_train_time

In [12]:
pred_start_time = time.time()
y_train_preds = [median] * y_train.shape[0]
train_pred_time = time.time() - pred_start_time

pred_start_time = time.time()
y_test_preds = [median] * y_test.shape[0]
test_pred_time = time.time() - pred_start_time

In [13]:
eval_train = evaluation_utils.get_evaluation(
    y_train, y_train_preds, regression=True, time=train_pred_time
)
eval_train["report"]["training_time"] = total_train_time
eval_valid = evaluation_utils.get_evaluation(
    y_test, y_test_preds, regression=True, time=test_pred_time
)
evaluation_report = {"Train": eval_train, "Test": eval_valid}

In [14]:
with open(os.path.join(model_output_path, "evaluation_report.json"), "w") as fp:
    json.dump(evaluation_report, fp, indent=2)

pprint.pprint(evaluation_report)

{'Test': {'report': {'MAE': 0.7175228056900788,
                     'MAPE': 3.3844834988527444,
                     'MSE': 0.8761725829978164,
                     'R^2': -0.006132299606122604,
                     'prediction_time': 0.0011377334594726562}},
 'Train': {'report': {'MAE': 0.737920512833352,
                      'MAPE': 4.480985036980794,
                      'MSE': 0.9888323749258312,
                      'R^2': -0.018265635563541327,
                      'prediction_time': 0.000980377197265625,
                      'training_time': 0.006369829177856445}}}
